# Environment Setting Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'D:\\Projects\\Stock Screener\\Stock-Screener-Agent'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 0. Ticker Parsing

In [6]:
from typing import Annotated 
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages 
from langgraph.checkpoint.memory import InMemorySaver 
from langchain_ollama import ChatOllama
from colorama import Fore 
from langgraph.prebuilt import ToolNode 

llm = ChatOllama(model='qwen2.5:14b')

In [ ]:
from langchain.tools import tool 
import yfinance as yf 

@tool
def simple_screener(screen_type:str, offset:int)-> str: 
    """Returns screened assets (stocks, funds, bonds) given popular criteria. 

    Args:
        screen_type: One of a default set of stock screener queries from yahoo finance. 
        aggressive_small_caps
        day_gainers
        day_losers
        growth_technology_stocks
        most_actives
        most_shorted_stocks
        small_cap_gainers
        undervalued_growth_stocks
        undervalued_large_caps
        conservative_foreign_funds
        high_yield_bond
        portfolio_anchors
        solid_large_growth_funds
        solid_midcap_growth_funds
        top_mutual_funds
      offset: the pagination start point

    Returns:
        The a JSON output of assets that meet the criteria
        """
    
    query = yf.PREDEFINED_SCREENER_QUERIES[screen_type]['query']
    result = yf.screen(query, offset=offset, size=5) 
     
    fields = ["symbol"] 
    output_data = []
    for stock_detail in result['quotes']: 
        details = {}
        for key, val in stock_detail.items(): 
            if key in fields: 
                details[key] = val 
        output_data.append(details) 
    
    return f"Stock Screener Results: {output_data}"

In [15]:
tools = [simple_screener]
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools)

In [ ]:
class State(dict): 
    messages: Annotated[list, add_messages]

def chatbot(state:State): 
    print(state['messages'])
    return {"messages":[llm_with_tools.invoke(state['messages'])]}

def router(state:State): 
    last_message = state['messages'][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls: 
        return "tools" 
    else: 
        return END 


graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", router)

memory = InMemorySaver() 
graph = graph_builder.compile(checkpointer=memory)

In [17]:
prompt = input("🤖 Pass your prompt here: " )
result = graph.invoke({"messages":[{"role":"user", "content":prompt}]}, config={"configurable":{"thread_id":1234}})
print(Fore.LIGHTYELLOW_EX + result['messages'][-1].content + Fore.RESET) 

[HumanMessage(content='top 5', additional_kwargs={}, response_metadata={}, id='926874d7-315b-453e-b4ba-0ca15fb522f4')]
[HumanMessage(content='top 5', additional_kwargs={}, response_metadata={}, id='926874d7-315b-453e-b4ba-0ca15fb522f4'), AIMessage(content="It seems like you're looking for the top 5 assets according to one of the screening criteria from Yahoo Finance. Could you please specify which type of screen (e.g., `day_gainers`, `most_actives`) and how many results you want beyond these five? If not specified, I'll default to fetching the top 5 small cap gainers as an example.\n\nWould you like to proceed with `small_cap_gainers`?\n", additional_kwargs={}, response_metadata={'model': 'qwen2.5:14b', 'created_at': '2025-10-17T12:04:41.4645785Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6625389900, 'load_duration': 3323557500, 'prompt_eval_count': 324, 'prompt_eval_duration': 187178200, 'eval_count': 121, 'eval_duration': 2891514800, 'model_name': 'qwen2.5:14b'}, id='ru

In [12]:
prompt = input("🤖 Pass your prompt here: " )
result = graph.invoke({"messages":[{"role":"user", "content":prompt}]}, config={"configurable":{"thread_id":1234}})
print(Fore.LIGHTYELLOW_EX + result['messages'][-1].content + Fore.RESET) 

[HumanMessage(content='Top 5 gainers', additional_kwargs={}, response_metadata={}, id='9eb6f95d-ef0b-40b9-90a8-dc283eaba38b'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:14b', 'created_at': '2025-10-17T11:28:35.9670911Z', 'done': True, 'done_reason': 'stop', 'total_duration': 35201784500, 'load_duration': 34021665300, 'prompt_eval_count': 326, 'prompt_eval_duration': 387404700, 'eval_count': 31, 'eval_duration': 686144000, 'model_name': 'qwen2.5:14b'}, id='run--eb81be83-2da0-4b8f-9821-eebe51aedea4-0', tool_calls=[{'name': 'simple_screener', 'args': {'offset': 0, 'screen_type': 'day_gainers'}, 'id': '597dd125-25e8-402c-9e9b-b62091825592', 'type': 'tool_call'}], usage_metadata={'input_tokens': 326, 'output_tokens': 31, 'total_tokens': 357}), ToolMessage(content="Stock Screener Results: [{'bid': 9.07, 'ask': 10.25, 'shortName': 'Ermenegildo Zegna N.V.', 'exchange': 'NYQ', 'fiftyTwoWeekHigh': 10.38, 'fiftyTwoWeekLow': 6.05, 'averageAnalystRating': '1.8

In [13]:
prompt = input("🤖 Pass your prompt here: " )
result = graph.invoke({"messages":[{"role":"user", "content":prompt}]}, config={"configurable":{"thread_id":1234}})
print(Fore.LIGHTYELLOW_EX + result['messages'][-1].content + Fore.RESET) 

[HumanMessage(content='Top 5 gainers', additional_kwargs={}, response_metadata={}, id='9eb6f95d-ef0b-40b9-90a8-dc283eaba38b'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:14b', 'created_at': '2025-10-17T11:28:35.9670911Z', 'done': True, 'done_reason': 'stop', 'total_duration': 35201784500, 'load_duration': 34021665300, 'prompt_eval_count': 326, 'prompt_eval_duration': 387404700, 'eval_count': 31, 'eval_duration': 686144000, 'model_name': 'qwen2.5:14b'}, id='run--eb81be83-2da0-4b8f-9821-eebe51aedea4-0', tool_calls=[{'name': 'simple_screener', 'args': {'offset': 0, 'screen_type': 'day_gainers'}, 'id': '597dd125-25e8-402c-9e9b-b62091825592', 'type': 'tool_call'}], usage_metadata={'input_tokens': 326, 'output_tokens': 31, 'total_tokens': 357}), ToolMessage(content="Stock Screener Results: [{'bid': 9.07, 'ask': 10.25, 'shortName': 'Ermenegildo Zegna N.V.', 'exchange': 'NYQ', 'fiftyTwoWeekHigh': 10.38, 'fiftyTwoWeekLow': 6.05, 'averageAnalystRating': '1.8